In [ ]:
!pip install syllables

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib_metadata 8.6.1
    Uninstalling importlib_metadata-8.6.1:
      Successfully uninstalled importlib_metadata-8.6.1


In [ ]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install pyphen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import spacy
from collections import Counter
import numpy as np
import re
import spacy
import syllables
import textstat
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import pyphen


In [ ]:
# Initialize models
nlp = spacy.load("en_core_web_sm")  # For syntactic analysis
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')  # For embedding similarity
pyphen_obj = pyphen.Pyphen(lang='en')  # For syllable counting

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Encapsulates 15 functions to compute various complexity metrics.
class TextComplexityAnalyzer:
    def __init__(self, text, nlp_model):
        self.text = text
        self.nlp = nlp_model
        self.doc = self.nlp(text)

    # Lexical Metrics
    def ttr_metric(self):
        tokens = re.findall(r'\w+', self.text.lower())
        return len(set(tokens)) / len(tokens) if tokens else 0

    def mltd_metric(self, threshold=0.72):
        tokens = re.findall(r'\w+', self.text.lower())
        total_tokens = len(tokens)
        if total_tokens == 0:
            return 0
        segments = 0
        token_count = 0
        types = set()
        for token in tokens:
            token_count += 1
            types.add(token)
            if len(types) / token_count <= threshold:
                segments += 1
                token_count = 0
                types = set()
        if token_count > 0:
            k = (len(types) / threshold) - token_count
            partial_seg = token_count / (token_count + k) if k > 0 else 1
            segments += partial_seg
        return total_tokens / segments if segments != 0 else total_tokens

    def entropy_metric(self):
        tokens = re.findall(r'\w+', self.text.lower())
        total = len(tokens)
        if total == 0:
            return 0
        freq = Counter(tokens)
        probs = np.array(list(freq.values())) / total
        return -np.sum(probs * np.log2(probs + 1e-12))

    # Word-Level Metrics
    def avg_word_length(self):
        tokens = re.findall(r'\w+', self.text)
        return sum(len(token) for token in tokens) / len(tokens) if tokens else 0

    def hapax_ratio(self):
        tokens = re.findall(r'\w+', self.text.lower())
        total = len(tokens)
        if total == 0:
            return 0
        freq = Counter(tokens)
        return sum(1 for count in freq.values() if count == 1) / total

    # Syntactic Metrics
    def tree_depth(self, token):
        if not list(token.children):
            return 1
        return 1 + max(self.tree_depth(child) for child in token.children)

    def avg_parse_depth(self):
        depths = [self.tree_depth(sent.root) for sent in self.doc.sents]
        return np.mean(depths) if depths else 0

    def max_parse_depth(self):
        depths = [self.tree_depth(sent.root) for sent in self.doc.sents]
        return max(depths) if depths else 0

    def subord_clause_ratio(self):
        subordinate_deps = {"advcl", "ccomp", "xcomp", "acl", "relcl", "csubj", "csubjpass"}
        total_sub = sum(sum(1 for token in sent if token.dep_ in subordinate_deps) for sent in self.doc.sents)
        total_clauses = sum(1 + sum(1 for token in sent if token.dep_ in subordinate_deps) for sent in self.doc.sents)
        return total_sub / total_clauses if total_clauses else 0

    def center_embed_depth(self):
        depths = [sum(1 for token in sent if token.dep_ == "relcl") for sent in self.doc.sents]
        return max(depths) if depths else 0

    def avg_dep_distance(self):
        distances = [abs(token.i - token.head.i) for token in self.doc if token.dep_ != "ROOT"]
        return np.mean(distances) if distances else 0

    def rec_dep_chain_len(self):
        def chain_length(token):
            lengths = [chain_length(child) for child in token.children if child.dep_ in {"ccomp", "xcomp"}]
            return 1 + max(lengths) if lengths else 1
        lengths = [chain_length(token) for token in self.doc if token.dep_ in {"ccomp", "xcomp"}]
        return max(lengths) if lengths else 0

    def avg_tunit_length(self):
        lengths = []
        for sent in self.doc.sents:
            words = [token for token in sent if token.is_alpha]
            lengths.append(len(words))
        return np.mean(lengths) if lengths else 0

    def clause_per_sent_ratio(self):
        subordinate_deps = {"advcl", "ccomp", "xcomp", "acl", "relcl", "csubj", "csubjpass"}
        ratios = []
        for sent in self.doc.sents:
            count = 1 + sum(1 for token in sent if token.dep_ in subordinate_deps)
            ratios.append(count)
        return np.mean(ratios) if ratios else 0

    def syn_struct_variance(self):
        subordinate_deps = {"advcl", "ccomp", "xcomp", "acl", "relcl", "csubj", "csubjpass"}
        counts = [1 + sum(1 for token in sent if token.dep_ in subordinate_deps) for sent in self.doc.sents]
        return np.std(counts) if counts else 0

    def rec_syn_depth(self):
        def recursive_depth(token, current=0):
            depths = [recursive_depth(child, current + 1) for child in token.children if child.dep_ in {"advcl", "ccomp", "xcomp", "relcl", "acl", "csubj", "csubjpass"}]
            return max([current] + depths) if depths else current
        depths = [recursive_depth(sent.root) for sent in self.doc.sents]
        return max(depths) if depths else 0

    # Compute All 15 Metrics
    def compute_all_metrics(self):
        return {
            "TTR": self.ttr_metric(),
            "MLTD": self.mltd_metric(),
            "Entropy": self.entropy_metric(),
            "AvgWordLength": self.avg_word_length(),
            "HapaxRatio": self.hapax_ratio(),
            "AvgParseDepth": self.avg_parse_depth(),
            "MaxParseDepth": self.max_parse_depth(),
            "SubordClauseRatio": self.subord_clause_ratio(),
            "CenterEmbedDepth": self.center_embed_depth(),
            "AvgDepDistance": self.avg_dep_distance(),
            "RecDepChainLen": self.rec_dep_chain_len(),
            "AvgTUnitLength": self.avg_tunit_length(),
            "ClausePerSentRatio": self.clause_per_sent_ratio(),
            "SynStructVar": self.syn_struct_variance(),
            "RecSynDepth": self.rec_syn_depth()
        }

In [ ]:
def enrich_csv(input_path: str, output_path: str, text_col: str = None):
    # 1. Load input CSV.
    #    If it has no header, pandas will auto‑name the first column "0"
    df = pd.read_csv(input_path)
    if text_col is None:
        text_col = df.columns[0]

    # 2. Load spaCy model once
    nlp = spacy.load("en_core_web_sm")

    # 3. Compute metrics for each row
    records = []
    for text in df[text_col].astype(str):
        analyzer = TextComplexityAnalyzer(text, nlp)
        rec = analyzer.compute_all_metrics()
        rec[text_col] = text
        records.append(rec)

    # 4. Build new DataFrame and write out
    out_df = pd.DataFrame(records)
    # reorder so text column comes first
    cols = [text_col] + [c for c in out_df.columns if c != text_col]
    out_df[cols].to_csv(output_path, index=False)
    print(f"Wrote enriched CSV to {output_path}")

if __name__ == "__main__":
    enrich_csv("full_event.csv", "full_event_complexity.csv")

Wrote enriched CSV to full_event_complexity.csv


In [ ]:
class ReadabilityAnalyzer:
    def __init__(self, text: str, nlp_model, sentence_model):
        self.text = text
        self.nlp = nlp_model
        self.sent_model = sentence_model

        # run spaCy once
        self.doc = self.nlp(text)
        self.sentences = [sent.text for sent in self.doc.sents]
        self.words = [tok.text.lower() for tok in self.doc if tok.is_alpha]

        # precompute embeddings & similarity matrix
        if self.sentences:
            self.embeddings = self.sent_model.encode(self.sentences)
            sim = cosine_similarity(self.embeddings)
            # upper‑triangle (no diagonal)
            idx = np.triu_indices(len(self.sentences), k=1)
            self._pairwise_sims = sim[idx]
        else:
            self.embeddings = np.array([])
            self._pairwise_sims = np.array([])

    # Vocabulary Simplicity
    def avg_word_length(self):
        return (sum(len(w) for w in self.words) / len(self.words)) if self.words else 0

    def avg_syllables(self):
        return (sum(syllables.estimate(w) for w in self.words) / len(self.words)) if self.words else 0

    def avg_age_of_acquisition(self):
        aoa = {"cat": 3, "dog": 3, "quantum": 12}
        vals = [aoa.get(w, 8) for w in self.words]
        return np.mean(vals) if vals else 0

    def avg_word_familiarity(self):
        fam = {"cat": 5, "dog": 5, "quantum": 2}
        vals = [fam.get(w, 3) for w in self.words]
        return np.mean(vals) if vals else 0

    def flesch_reading_ease(self):
        return textstat.flesch_reading_ease(self.text)

    def gunning_fog(self):
        return textstat.gunning_fog(self.text)

    # Discourse Cohesion
    def lexical_overlap(self):
        overlaps = []
        for a, b in zip(self.sentences, self.sentences[1:]):
            w1 = set(re.findall(r'\w+', a.lower()))
            w2 = set(re.findall(r'\w+', b.lower()))
            denom = len(w1 | w2) + 1e-12
            overlaps.append(len(w1 & w2) / denom)
        return np.mean(overlaps) if overlaps else 0

    def connective_density(self):
        conn = {"because", "however", "therefore", "thus", "although"}
        return (sum(1 for w in self.words if w in conn) / len(self.words)) if self.words else 0

    def referential_cohesion(self):
        overlap = 0
        for prev, curr in zip(self.sentences, self.sentences[1:]):
            ents1 = {e.text.lower() for e in self.nlp(prev).ents}
            ents2 = {e.text.lower() for e in self.nlp(curr).ents}
            overlap += len(ents1 & ents2)
        return (overlap / len(self.sentences)) if self.sentences else 0

    # Embedding-Based
    def semantic_similarity(self):
        return np.mean(self._pairwise_sims) if self._pairwise_sims.size else 0

    def avg_cosine_similarity(self):
        return self.semantic_similarity()

    def variance_of_embedding_distances(self):
        return np.var(self._pairwise_sims) if self._pairwise_sims.size else 0

    # All metrics
    def compute_all_metrics(self):
        return {
            "avg_word_length":           self.avg_word_length(),
            "avg_syllables":             self.avg_syllables(),
            "avg_age_of_acquisition":    self.avg_age_of_acquisition(),
            "avg_word_familiarity":      self.avg_word_familiarity(),
            "flesch_reading_ease":       self.flesch_reading_ease(),
            "gunning_fog":               self.gunning_fog(),
            "lexical_overlap":           self.lexical_overlap(),
            "connective_density":        self.connective_density(),
            "referential_cohesion":      self.referential_cohesion(),
            "semantic_similarity":       self.semantic_similarity(),
            "avg_cosine_similarity":     self.avg_cosine_similarity(),
            "variance_of_embedding_distances": self.variance_of_embedding_distances(),
        }

In [ ]:
def generate_readability_csv(input_path: str,
                             output_path: str,
                             text_col: str = None):
    # 1. Load your input CSV
    df = pd.read_csv(input_path)
    if text_col is None:
        text_col = df.columns[0]  # assume first column

    # 2. Initialize models once
    nlp_model = spacy.load("en_core_web_sm")
    sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

    # 3. Compute readability metrics
    rows = []
    for txt in df[text_col].astype(str):
        analyzer = ReadabilityAnalyzer(txt, nlp_model, sentence_model)
        metrics = analyzer.compute_all_metrics()
        metrics[text_col] = txt
        rows.append(metrics)

    # 4. Build and write out the new DataFrame
    out_df = pd.DataFrame(rows)
    cols = [text_col] + [c for c in out_df.columns if c != text_col]
    out_df[cols].to_csv(output_path, index=False)
    print(f"Wrote readability scores to {output_path}")

if __name__ == "__main__":
    generate_readability_csv("full_event.csv", "full_event_readability.csv")

Wrote readability scores to full_event_readability.csv
